In [1]:
import pandas as pd
import numpy as np
import math

import irc6_2007 as irc6

from Cross import Cross_section,cables, expansion_calc
from bridge_specs import box
from allinput import main_input

In [2]:
df=pd.read_excel('data/span.xlsx',index_col=None,header=None)
fck = 35 #input grade of concrete
span = int(df.iloc[2])
no = int(df.iloc[1])
sc=[(j)/(no-1)*50 for j in range(no)]

***required:***
1. list of sections with cable profile
2. BM due to permanent dead load
3. BM due to superimposed dead load (without surfacing)
4. BM due to surfacing
5. BM due to live load (max of all vehicle loads)

**defining sections**

main section

In [3]:

"""INPUTS SECTION"""

"""FOR CROSS SECTION"""
length=[]
height=[]


#####################################################################################################################

"""GUI INPUT OF BRIDGE SECTION LENGTHS AND HEIGHTS"""

if False:
    main_input()

df=pd.read_excel('data/box.xlsx',index_col=None,header=None)


height=df.values.tolist()[0]
length=df.values.tolist()[1]


section=Cross_section(length,height)
cable_no = 14 #input no of cables
cable = cables(fck,cable_no,section)

Generating intermediate sections

In [4]:
sections = []
for i in range(no):  
    section1=Cross_section(length,height)
    section1.expansion_width=expansion_calc(span,sc[i],cable)
    section1.cableprop=cable.cablepos(sc[i],50)
    sections.append(section1)

In [5]:
# endsec = sections[0]
# endsec.cable_prop

# midsec = sections[4]
# midsec.cable_prop, endsec.Centroid

**Importing load**

In [8]:
df = pd.read_excel('outputs/Moments.xlsx', index_col=0)

#bm multipied by distribution factor
bm_pdl = np.array(df['Dead Load'])
bm_odl = np.array(df['Other Loads'])
bm_surf = np.array(df['Surface Loads'])
bm_pedl = np.array(df['Pedestrian Load'])

dfl = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1]) #live load

**live load calculation**

Impact factor

In [9]:
vehicles = [irc6.ll_A, irc6.ll_70R, irc6.ll_70RT]
IF = [irc6.impact(i.name, span) for i in vehicles]
arr_if = np.array(IF)
arr_with_if = np.array(IF)+1 #including impact factor

distribution factor

In [10]:
dfe = pd.read_excel('outputs/max_e.xlsx', index_col=0)

ecc = dfe['MaxEccentricity'] #eccentricity

def df(ecc, webcc):
    df = (webcc/2+ecc)/webcc
    return round(df, 3)

webcc = 3 #input web center/center distance
distf = [df(i, webcc) for i in ecc]
distf

[1.067, 0.5, 0.635, 0.617]

bm for ll combinations

In [15]:
arr_maxbm = dfl.loc['MaxBM'].to_numpy() # max bm due to each veh
arr_maxbmif = arr_maxbm*arr_with_if[:, np.newaxis] # max bm including impact due to ll

arr_ll_comb = dfe.drop(columns='MaxEccentricity').to_numpy() # ll combinations in array

all_ll = arr_ll_comb.dot(arr_maxbmif) # ll with impact for each combination
all_unfactored_ll = all_ll #without dist factor
# all_unfactored_ll = all_ll*np.array(distf)[:, np.newaxis] # distributed bm due to live load including impact (bm*if*df)

bm_ll = np.amax(all_unfactored_ll, axis=0)
bm_ll

array([    0.      ,  5250.7152  ,  8922.6336  , 11142.723712,
       12024.9024  , 11619.922688,  9417.0208  ,  5428.332288,
           0.      ])

### Cable profile

In [16]:
# def cable_profile(end_cable_pos, mid_cable_pos, span_length, division=8):
#     cable_profile = []
#     x = [(j)/division*span_length for j in range(division+1)]
#     for i in x:
#         cable_pos_x = []
#         for j in range(len(end_cable_pos)):
#             h = end_cable_pos[j][1]-mid_cable_pos[j][1]
#             a = end_cable_pos[j][1]
#             y = a + 4*h*i/span_length - 4*h*i**2/span_length**2
#             cable_pos_x.append((end_cable_pos[j][0], y))
#         cable_profile.append(cable_pos_x)
#     return cable_profile

### Eccentricity of cables

In [17]:
def eccentricity(cable_positions, centroid):
    cable_areaxpos_sum = 0
    for i in cable_positions:
        cable_areaxpos_sum+=(i[1]-centroid)
    return cable_areaxpos_sum/len(cable_positions)

### Stress

In [18]:
list_of_sections = sections
prestressing_force = 2730 #input
prestressing_force_total = prestressing_force*cable_no
# prestressing_force_withloss = 0.75*prestressing_force

In [19]:
def tau(section, bm):
    tau_top = bm*section.ymin/section.I[0]
    tau_bot = -bm*section.ymax/section.I[0]
    return (tau_top, tau_bot)

presetressing stress

In [20]:
#due to eccentricity
bm_pre = [prestressing_force_total*eccentricity(i.cableprop, i.Centroid[1]) for i in list_of_sections] #bm due to full prestressing force

list_tau_pre = [tau(list_of_sections[index], bm_pre[index]) for index in range(len(list_of_sections))] #stress due to full prestressing force

#axial
list_tau_ax = [prestressing_force_total/sum(i.section_area) for i in list_of_sections]

list_tau_preTotal = [(list_tau_pre[index][0] + list_tau_ax[index], list_tau_pre[index][1] + list_tau_ax[index]) for index in range(len(list_of_sections))]

In [21]:
list_tau_preTotal

[(2698.9277713847255, 9433.945640801687),
 (2144.793996825409, 10591.4480723289),
 (1748.9841578544688, 11418.235523419768),
 (1511.4982544719041, 11914.307994074286),
 (1432.3362866777165, 12079.66548429246),
 (1511.4982544719041, 11914.307994074286),
 (1748.9841578544683, 11418.235523419768),
 (2144.793996825409, 10591.4480723289),
 (2698.9277713847255, 9433.945640801687)]

stress due to ded load

In [22]:
list_tau_DL = [tau(list_of_sections[index], bm_pdl[index]) for index in range(len(list_of_sections))]

In [23]:
list_tau_DL

[(0.0, -0.0),
 (2070.2313492650756, -4324.403114579835),
 (3548.968027311557, -7413.262482136858),
 (4436.210034139446, -9266.578102671074),
 (4731.957369748742, -9884.349976182479),
 (4436.210034139446, -9266.578102671074),
 (3548.968027311557, -7413.262482136858),
 (2070.2313492650756, -4324.403114579835),
 (0.0, -0.0)]

stress due to surfacing

In [24]:
list_tau_surf = [tau(list_of_sections[index], bm_surf[index]) for index in range(len(list_of_sections))]

In [25]:
list_tau_surf

[(0.0, -0.0),
 (90.70642143084049, -189.47212420832363),
 (155.49672245286942, -324.8093557856976),
 (194.37090306608678, -406.01169473212207),
 (207.32896327049255, -433.07914104759686),
 (194.37090306608678, -406.01169473212207),
 (155.49672245286942, -324.8093557856976),
 (90.70642143084049, -189.47212420832363),
 (0.0, -0.0)]

stress due to other load

In [26]:
list_tau_ODL = [tau(list_of_sections[index], bm_odl[index]) for index in range(len(list_of_sections))]

In [27]:
list_tau_ODL

[(0.0, -0.0),
 (137.43397186490984, -287.07897607321763),
 (235.6010946255597, -492.1353875540873),
 (294.5013682819496, -615.1692344426091),
 (314.13479283407963, -656.1805167387831),
 (294.5013682819496, -615.1692344426091),
 (235.6010946255597, -492.1353875540873),
 (137.43397186490984, -287.07897607321763),
 (0.0, -0.0)]

stress due to live load

In [28]:
list_tau_LL = [tau(list_of_sections[index], bm_ll[index]) for index in range(len(list_of_sections))]

In [29]:
arr_stress_top = np.array([
    list(zip(*list_tau_DL))[0], 
    list(zip(*list_tau_ODL))[0], 
    list(zip(*list_tau_surf))[0], 
    list(zip(*list_tau_LL))[0], 
    list(zip(*list_tau_preTotal))[0]
])

arr_stress_bot = np.array([
    list(zip(*list_tau_DL))[1], 
    list(zip(*list_tau_ODL))[1], 
    list(zip(*list_tau_surf))[1], 
    list(zip(*list_tau_LL))[1], 
    list(zip(*list_tau_preTotal))[1]
])

**Case 1: Rare combination I**

RI = DL + PS@10%loss

In [30]:
psf_RI = np.array([1, 0, 0, 0, 0.9]) #partial safety factor [DL, ODL, surf, LL, Ps] irc:6-2007 table B3

tauTop_RI = psf_RI.dot(arr_stress_top) #resultant pressure top

tauBot_RI = psf_RI.dot(arr_stress_bot) #resultant pressure bottom

tauTop_RI, tauBot_RI

(array([2429.03499425, 4000.54594641, 5123.05376938, 5796.55846316,
        6021.06002776, 5796.55846316, 5123.05376938, 4000.54594641,
        2429.03499425]),
 array([8490.55107672, 5207.90015052, 2863.14948894, 1456.299092  ,
         987.34895968, 1456.299092  , 2863.14948894, 5207.90015052,
        8490.55107672]))

Allowable stresses

In [31]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.48*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-2771, 16800)

check

In [32]:
if all(tauTop_RI>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_RI<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok


**Case 2: Rare combination II**

RII = DL + ODL + 1.2*Surf + 0.75*LL + PS@25%loss

In [33]:
psf_RII = np.array([1, 1, 1.2, 0.75, 0.75]) #partial safety factor [DL, ODL, surf, LL, PS] irc:6-2007 table B3

tauTop_RII = psf_RII.dot(arr_stress_top) #resultant pressure top

tauBot_RII = psf_RII.dot(arr_stress_bot) #resultant pressure bottom

tauTop_RII, tauBot_RII

(array([2024.19582854, 4229.61910876, 5800.36355173, 6743.79258401,
        7066.51271338, 6771.46731659, 5829.03510016, 4239.91985469,
        2024.19582854]),
 array([ 7075.4592306 ,  2468.66042749,  -812.38937585, -2783.07127429,
        -3457.18527276, -2840.87964426,  -872.27994478,  2447.14371256,
         7075.4592306 ]))

Allowable stresses

In [34]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.36*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-2771, 12600)

check

In [35]:
if all(tauTop_RII>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_RII<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok


**Case 2: Rare combination II**

RII = DL + ODL + 1.2*Surf + LL + PS@25%loss

In [36]:
psf_RII = np.array([1, 1, 1.2, 1, 0.75]) #partial safety factor [DL, ODL, surf, LL, PS] irc:6-2007 table B3

tauTop_RII = psf_RII.dot(arr_stress_top) #resultant pressure top

tauBot_RII = psf_RII.dot(arr_stress_bot) #resultant pressure bottom

tauTop_RII, tauBot_RII

(array([2024.19582854, 4331.12263686, 5972.85029988, 6959.19671969,
        7298.97057334, 6996.09636313, 6011.07903112, 4344.85696477,
        2024.19582854]),
 array([ 7075.4592306 ,  2256.63476514, -1172.68834978, -3233.01824064,
        -3942.75491402, -3310.09606727, -1252.54244168,  2227.94581189,
         7075.4592306 ]))

Allowable stresses

In [37]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.46*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-2771, 16100)

check

In [38]:
if all(tauTop_RII>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_RII<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok


**Case 3: Permanent combination**

RII = DL + ODL + 1.2*Surf + 0.75*LL + PS@25%loss

In [39]:
psf_P = np.array([1, 1, 1.2, 0.75, 0.75]) #partial safety factor [DL, ODL, surf, LL, PS] irc:6-2007 table B3

tauTop_P = psf_P.dot(arr_stress_top) #resultant pressure top

tauBot_P = psf_P.dot(arr_stress_bot) #resultant pressure bottom

tauTop_P, tauBot_P

(array([2024.19582854, 4229.61910876, 5800.36355173, 6743.79258401,
        7066.51271338, 6771.46731659, 5829.03510016, 4239.91985469,
        2024.19582854]),
 array([ 7075.4592306 ,  2468.66042749,  -812.38937585, -2783.07127429,
        -3457.18527276, -2840.87964426,  -872.27994478,  2447.14371256,
         7075.4592306 ]))

Allowable stresses

In [40]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.36*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-2771, 12600)

check

In [41]:
if all(tauTop_P>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_P<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok


**Case 4: Quasi-Permanent**

QP = DL + ODL + PS@25%loss

In [42]:
psf_QP = np.array([1, 1, 0, 0, 0.75]) #partial safety factor [DL, ODL, surf, LL, PS] irc:6-2007 table B3

tauTop_QP = psf_QP.dot(arr_stress_top) #resultant pressure top

tauBot_QP = psf_QP.dot(arr_stress_bot) #resultant pressure bottom

tauTop_QP, tauBot_QP

(array([2024.19582854, 3816.26081875, 5096.30724033, 5864.33509328,
        6120.34437759, 5864.33509328, 5096.30724033, 3816.26081875,
        2024.19582854]),
 array([ 7075.4592306 ,  3332.10396359,   658.27877287,  -946.01634156,
        -1480.7813797 ,  -946.01634156,   658.27877287,  3332.10396359,
         7075.4592306 ]))

Allowable stresses

In [43]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.36*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-2771, 12600)

check

In [44]:
if all(tauTop_QP>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_QP<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok
